In [1]:
# Reference : https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances_chunked.html#sklearn.metrics.pairwise_distances_chunked
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances_chunked

In [20]:
#df = np.random.RandomState(0).rand(300000, 8)
#df
df = pd.read_csv("data_cust_prod_matrix.csv") #.head(30000) #runs fine with 20000
#df.head()

In [21]:
prod_df = df.iloc[:,1:9] # slecting the product columns excusing user id
prod_mat = prod_df.as_matrix(columns=None) # converting the dataframe into matrix 

C:\Users\u004637\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [22]:
item_similarity = next(pairwise_distances_chunked(prod_mat.T, metric='cosine'))

print("item_similarity : " + str(item_similarity.shape))

item_similarity : (8, 8)


In [33]:
#colors = ["red", "green", "blue", "purple"]
def reduce_func1(D_chunk, start):
    #neigh = [np.flatnonzero(d < 3) for d in D_chunk]
    #avg_dist = (D_chunk * (D_chunk < r)).mean(axis=1)
    neigh = 2
    avg_dist = 3
    return neigh, avg_dist

def reduce_func():
    x=1
    return 100

i = 0
while i < 2:
    user_similarity = next(pairwise_distances_chunked(prod_mat, metric='cosine', reduce_func=reduce_func))
    print("user_similarity : " + str(user_similarity.shape))
    i += 1

TypeError: reduce_func() takes 0 positional arguments but 2 were given

In [58]:
""" Function for recommenation matix generation """

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #We use np.newaxis so that mean_user_rating has same format as s_ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

In [59]:

""" running function against the data matrix """

#user_prediction = predict(prod_mat, user_similarity, type='user')
item_prediction = predict(prod_mat, item_similarity, type='item')

#print("user_prediction : " + str(user_prediction.shape))
print("item_prediction : " + str(item_prediction.shape))

item_prediction : (30000, 8)


In [60]:
'''
user based method accuracy. 
1-MAE is used as accuracy as we can compare and 
also the out put of recommendation ening and input is binary.
This is appropriate metric.

min max transformation. 0 to 1 range

'''

scaled_user_pred = (user_prediction - user_prediction.min()) / (user_prediction.max() - user_prediction.min())
mean = scaled_user_pred.mean()
bi_user_pred = (scaled_user_pred > mean).astype(int)


'''
mean_real = prod_mat.mean()
bi_real = (prod_mat > mean_real).astype(int) # any value greater than mean is flagged as 1 else 0
'''
 
from sklearn.metrics import mean_absolute_error

Accuracy_user_sim = 1- mean_absolute_error(prod_mat, bi_user_pred)


'''
item based method accuracy

'''

scaled_item_pred = (item_prediction - item_prediction.min()) / (item_prediction.max() - item_prediction.min())
mean = scaled_item_pred.mean()
bi_item_pred = (scaled_item_pred > mean).astype(int)

'''
mean_real = prod_mat.mean()
bi_real = (prod_mat > mean_real).astype(int)
'''

ValueError: Found input variables with inconsistent numbers of samples: [30000, 1000]

In [ ]:
from sklearn.metrics import mean_absolute_error

Accuracy_item_sim = 1- mean_absolute_error(prod_mat, bi_item_pred)

print(Accuracy_item_sim)

print(Accuracy_user_sim)